In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, FFMpegWriter
from scipy.ndimage import uniform_filter
import matplotlib.gridspec as gridspec

# --- CONFIGURAÇÃO DO APOCALIPSE ---
GRID_SIZE = 500
EPOCHS = 150 

LEARNING_RATE = 0.5 
QTD_RUIDO = 0.001

np.random.seed(666) # Número do Azar para garantir o caos

# 1. ENTROPIA MÁXIMA (Mudança Solicitada)
# Em vez de cinza médio (random.rand), usamos BINÁRIO (0 ou 1).
# Isso maximiza a variância inicial (0.25 vs 0.08 do uniforme).
# É o contraste absoluto.
data = np.random.choice([0.0, 1.0], size=(GRID_SIZE, GRID_SIZE))

# --- PREPARAÇÃO DA VISUALIZAÇÃO ---
fig = plt.figure(figsize=(16, 9), facecolor='#121212') # Dark Mode Profundo
gs = gridspec.GridSpec(2, 2, width_ratios=[1, 1], height_ratios=[2, 1])

# Subplot 1: A Realidade (Imagem/Dados)
ax_img = fig.add_subplot(gs[0, 0])
im = ax_img.imshow(data, cmap='gray', vmin=0, vmax=1) # Mudei pra Gray pra ficar mais dramático
ax_img.set_title("A Realidade (Binária & Crua)", color='white', fontsize=14, fontname='Monospace')
ax_img.axis('off')

# Subplot 2: Loss Landscape (O Poço da Mediocridade)
ax_loss = fig.add_subplot(gs[0, 1], projection='3d')
ax_loss.set_facecolor('#121212')
ax_loss.set_title("O Poço da Convergência (Loss)", color='white', fontsize=14, fontname='Monospace')

# Criando o "Bowl" (bacia) mais acentuado
X_mesh = np.linspace(-2, 2, 30)
Y_mesh = np.linspace(-2, 2, 30)
X_mesh, Y_mesh = np.meshgrid(X_mesh, Y_mesh)
Z_mesh = np.log(X_mesh**2 + Y_mesh**2 + 1) # Log-Loss visual para combinar com o tema
ax_loss.plot_wireframe(X_mesh, Y_mesh, Z_mesh, color='#555555', alpha=0.4, rstride=2, cstride=2)
ax_loss.axis('off')

loss_dot, = ax_loss.plot([], [], [], 'o', color='#00ff00', markersize=12, markeredgecolor='white', label='Estado da Arte')

# Subplot 3: O Gráfico Logarítmico (Mudança Solicitada)
ax_stats = fig.add_subplot(gs[1, :])
ax_stats.set_facecolor('#1e1e1e')
ax_stats.set_title("Decaimento da Entropia (Escala Logarítmica)", color='#ff0055', fontsize=12, fontweight='bold')
ax_stats.set_xlim(0, EPOCHS)

# ESCALA LOGARÍTMICA ATIVADA
ax_stats.set_yscale('log') 
ax_stats.set_ylim(1e-4, 1.0) # Focando no detalhe infinitesimal
ax_stats.grid(True, which="both", ls="-", color='#333333')

ax_stats.tick_params(axis='x', colors='white')
ax_stats.tick_params(axis='y', colors='white')
ax_stats.set_ylabel("Variância (log)", color='white')
ax_stats.set_xlabel("Gerações de IA (Tempo)", color='white')

line_entropy, = ax_stats.plot([], [], color='#00ffff', lw=2, label='Criatividade Restante')
ax_stats.legend(loc='upper right', facecolor='#1e1e1e', labelcolor='white')

history_variance = []
history_epochs = []

# --- O MOTOR DA SIMULAÇÃO ---

def update(frame):
    global data
    
    # 1. O INBREEDING (Convolução)
    # A média dos vizinhos destrói o contraste binário
    smoothed = uniform_filter(data, size=3, mode='reflect')
    
    # Atualização exponencial
    data = data * (1 - LEARNING_RATE) + smoothed * LEARNING_RATE
    
    # Pequeno ruído para mostrar que nem a aleatoriedade salva
    noise = np.random.normal(0, QTD_RUIDO, size=data.shape)
    data = data + noise
    
    # 2. MÉTRICAS
    current_variance = np.var(data)
    history_variance.append(current_variance)
    history_epochs.append(frame)

    # 3. VISUALIZAÇÃO
    im.set_data(data)
    
    line_entropy.set_data(history_epochs, history_variance)
    
    # Movimento no gráfico 3D (Espirlando para o ralo)
    # Usamos log na variância para mapear a posição, senão chega no centro rápido demais
    radius = np.log(current_variance * 100 + 1) * 0.8
    angle = frame * 0.15
    x_pos = radius * np.cos(angle)
    y_pos = radius * np.sin(angle)
    z_pos = np.log(x_pos**2 + y_pos**2 + 1)
    
    loss_dot.set_data([x_pos], [y_pos])
    loss_dot.set_3d_properties([z_pos])

    # TEXTOS DE NARRAÇÃO PARA O VIDEO
    if frame == 1:
        ax_img.set_xlabel("INÍCIO: Caos Puro (Bits 0 e 1)", color='#00ff00', fontsize=14)
    elif frame == 30:
        ax_img.set_xlabel("Fase 1: O Cinza Avança", color='yellow', fontsize=14)
    elif frame == 80:
        ax_img.set_xlabel("Fase 2: Morte Térmica", color='orange', fontsize=14)
    elif frame == 140:
        ax_img.set_xlabel("FIM: O Bege Eterno", color='red', fontsize=14, fontweight='bold')

    return im, line_entropy, loss_dot

# --- SALVAR ---
ani = FuncAnimation(fig, update, frames=EPOCHS, interval=40, blit=False)

print("Gerando o apocalipse visual em log-scale...")
writer = FFMpegWriter(fps=10, metadata=dict(artist='Contra a Maquina'), bitrate=3000)
ani.save("entropia_logaritmica.mp4", writer=writer)
print("Salvo: entropia_logaritmica.mp4")

plt.show()